# Konvoluční sítě pro klasifikaci

Úkolem cvičení je upravit tento notebook a dosáhnout co nejlepšího možného validačního skóre na datasetu CIFAR-10. Viz nápovědu a možné směry úprav v komentářích u jednotlivých buněk. Klasifikaci obrázků pomocí konvolučních sítí v PyTorch popisuje notebook [pytorch-convnets](lectures/pytorch-convnets.ipynb). 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

import ans

%load_ext autoreload
%autoreload 2

## Načtení CIFAR10

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# zlepsi predzpracovani a jine augmentace skore?
train_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=train_transform)
trainset

In [ ]:
# augmentaci lze provadet i v testovacim rezimu
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=test_transform)
testset

In [ ]:
# pomuze jina batch_size?
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

## Kritérium

In [ ]:
# muze zlepsit skore napr. SVM?
crit = nn.CrossEntropyLoss()
crit

## Definice konvoluční sítě

Síť definujte následující třídou `Convnet`. Architekturu síťě můžete navrhnout sami ručně, nebo lze použít jeden z existujících modelů, např. z katalogu `torchvision.models`. Není povoleno použít předtrénovaný model, tj. s vahami již natrénovanými na jiných datasetech, např. ImageNet.

In [ ]:
class Convnet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1, bias=False)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        self.fc1 = nn.Linear(32 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        # prvni konv. vrstva
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        # druha konv. vrstva
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

## Trénování a validace

In [ ]:
# pokud vytvorime novou sit, vyresetujeme i statistiky
convnet = Convnet()
stats = ans.Stats()

In [ ]:
# chceme-li vyuzit GPU, jinak nevolat
convnet.to('cuda')

In [ ]:
for name, par in convnet.named_parameters():
    print(name, par.shape, par.numel())

In [ ]:
# s novou siti musime obnovit i seznam parametru pro optimizer
# lepsich vysledku obvykle dosahuje SGD s momentum nebo napr. metoda Adam
optimizer = torch.optim.SGD(cnet.parameters(), lr=0.01)optimizer
optimizer

In [ ]:
# jaky vliv ma pocet epoch?
# zkuste postupne menit learning rate (optimizer.param_groups[0]['lr'] = ...)
# nebo pomoci scheduleru (https://pytorch.org/docs/master/optim.html#how-to-adjust-learning-rate)
for ep in range(1):
    stats.new_epoch()
    ans.train_pytorch(convnet, crit, train_loader, optimizer, stats)
    ans.validate_pytorch(convnet, crit, test_loader, stats)

In [ ]:
stats.plot_by_batch(block_len=10)

In [ ]:
stats.plot_by_epoch()

In [ ]:
stats.best_results()

## Predikce na testovacím obrázku

In [ ]:
rgb_test = cv2.imread('./data/happy-green-frog.jpg')[..., ::-1]

In [ ]:
ans.predict_and_show(cv2.resize(rgb_test, (32, 32)), convnet, test_transform, classes=classes)